In [ ]:
import numpy as np
import pandas as pd
import string
import re
import nltk
import gensim

from gensim import corpora
from gensim.models import LdaModel
from nltk.tokenize import word_tokenize
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from gensim import similarities
from gensim import models
from nltk.stem.porter import *

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/vannguyen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [1]:
import nltk
from nltk.tokenize import word_tokenize

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [4]:
df = pd.read_excel("amazon_review_processed_full.xlsx")
df.head(5)

,Review Model,Retailer,Review date,Review name,Review rating,Review title,Review Content,Verified Purchase or not,People_find_helpful,vine or not,...,title_word_count,content_char_count,title_char_count,content_avg_word,title_avg_word,content_stopwords,title_stopwords,Brand,Full review,ID
0,Canon Pixma TS6420a,Amazon,2023-08-10,Ernest Birkholz,5,work great,easi instal work great,True,0.0,False,...,2,38,11,3.875000,5.000000,3,0,Canon,work great easi instal work great,1
1,HP OfficeJet Pro 9015e,Amazon,2022-06-04,mattey,3,spunki mid size,slower speed im use old model 8600 new 9015e s...,True,0.0,False,...,4,460,23,4.835443,5.000000,19,0,HP,spunki mid size slower speed im use old model ...,2
2,Canon PIXMA MG3620,Amazon,2023-03-15,Maria D,4,NaN,i,True,0.0,False,...,2,7,12,3.000000,5.500000,0,0,Canon,i,3
3,Epson - ET-3830,Amazon,2022-11-17,Ryan H,5,shag know he talk,yup slam upgrad paid littl front equival throw...,True,12.0,False,...,6,469,34,4.595238,4.833333,31,2,Epson,shag know he talk yup slam upgrad paid littl f...,4
4,HP ENVY 6055e,Amazon,2022-04-09,Sam,3,user friendli,would think someth simpl reconnect wifi would ...,True,0.0,False,...,3,861,17,4.387500,5.000000,71,0,HP,user friendli would think someth simpl reconne...,5


In [3]:
pip install openpyxl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 6.4 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Tokenisation (Full Review)
df['Tokenized Full review'] = df['Full review'].apply(lambda x: word_tokenize(str(x)) if isinstance(x, str) else [])
df[['Full review', 'Tokenized Full review']].head(5)

,Full review,Tokenized Full review
0,work great easi instal work great,"[work, great, easi, instal, work, great]"
1,spunki mid size slower speed im use old model ...,"[spunki, mid, size, slower, speed, im, use, ol..."
2,i,[i]
3,shag know he talk yup slam upgrad paid littl f...,"[shag, know, he, talk, yup, slam, upgrad, paid..."
4,user friendli would think someth simpl reconne...,"[user, friendli, would, think, someth, simpl, ..."


In [6]:
# Create a dictionary from the tokenized content
dictionary = corpora.Dictionary(df['Tokenized Full review'])

# Create a corpus (bag of words) from the tokenized content
corpus = [dictionary.doc2bow(text) for text in df['Tokenized Full review']]
# To find the similarity scores, create a reverse index
Index = similarities.SparseMatrixSimilarity(corpus, len(dictionary))

# Create a TFIDF reveset index
TFIDF = models.TfidfModel(corpus)
corpus_TFIDF = [TFIDF[vec] for vec in corpus]
Index_TFIDF = similarities.SparseMatrixSimilarity(corpus_TFIDF, len(dictionary))

In [ ]:
# A Query string
query = input("What would you like to search for? ")
qList = query.split()
qLower = [w.lower() for w in qList]

# Stem it 
stemmer = PorterStemmer()
qStemmed = [stemmer.stem(w) for w in qLower]

# Create a query vector using the same dicitonary as the corpus
qVector = dictionary.doc2bow(qStemmed)

# Get its TFIDF from the same model as the corpus
qVectorTFIDF = TFIDF[qVector]
# print(qVector, qVectorTFIDF)

# Get the similaries from the two indexes (raw and TFIDF)
simRaw = Index[qVector]
simTFIDF = Index_TFIDF[qVectorTFIDF]

# Set display options to show the entire review content
pd.set_option('display.max_colwidth', None)

# Create a DataFrame to store the results
results_raw = pd.DataFrame({
    'Document': range(len(df)),
    'Raw Similarity Score': simRaw
})

results_tfidf = pd.DataFrame({
    'Document': range(len(df)),
    'TFIDF Similarity Score': simTFIDF
})

# Add the Similarity Score as a new column to the original DataFrame
df['Raw Similarity Score'] = results_raw['Raw Similarity Score']
df['TFIDF Similarity Score'] = results_tfidf['TFIDF Similarity Score']

# Sort the DataFrame by Raw Similarity Score in descending order
df_sorted_raw = df.sort_values(by='Raw Similarity Score', ascending=False)
df_sorted_tfidf = df.sort_values(by='TFIDF Similarity Score', ascending=False)

# Display the sorted DataFrame

selected_columns = ["Review Model", "Review date", "Review rating", "Original title", "Original review", "TFIDF Similarity Score"]

# Display the first 10 rows of the selected columns
df_sorted_tfidf_selected = df_sorted_tfidf[selected_columns]

df_sorted_tfidf_selected.head(10)


In [ ]:
pip install streamlit

In [ ]:
!streamlit run test_app.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8504
  Network URL: http://192.168.1.237:8504

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
2023-10-14 21:22:04.647 Uncaught app exception
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/streamlit/runtime/scriptrunner/script_runner.py", line 552, in _run_script
    exec(code, module.__dict__)
  File "/Users/vannguyen/HCAP Dropbox/Van Nguyen/Mac/Documents/GitHub/CustomerProductReviews/test_app.py", line 3, in <module>
    from gensim import corpora, similarities, models
ModuleNotFoundError: No module named 'gensim'


In [ ]:
pip install nltk


In [ ]:
pip install gensim


In [ ]:
pip install scikit-learn